In [1]:
import os
import pandas as pd
import numpy as np
#Pull NPS API data
import requests
import json
#Database connectivity
from sqlalchemy import create_engine

#Import api key and values for database connectivity
from config import (nps_api_key, user, password, host, port, database)


In [2]:
path_xlsx = os.getcwd()
files_xlsx = os.listdir(path_xlsx+"/data")

filepath_xlsx = [];
for f in files_xlsx:
    if f[-4:] == 'xlsx':
        filepath_xlsx.append(f'data/{f}');
                    
filepath_xlsx

['data/Current Year Monthly and Annual Summary Report (1979 - Present) (Apr).xlsx',
 'data/Current Year Monthly and Annual Summary Report (1979 - Present) (Aug).xlsx',
 'data/Current Year Monthly and Annual Summary Report (1979 - Present) (Dec).xlsx',
 'data/Current Year Monthly and Annual Summary Report (1979 - Present) (Feb).xlsx',
 'data/Current Year Monthly and Annual Summary Report (1979 - Present) (Jan).xlsx',
 'data/Current Year Monthly and Annual Summary Report (1979 - Present) (Jul).xlsx',
 'data/Current Year Monthly and Annual Summary Report (1979 - Present) (Jun).xlsx',
 'data/Current Year Monthly and Annual Summary Report (1979 - Present) (Mar).xlsx',
 'data/Current Year Monthly and Annual Summary Report (1979 - Present) (May).xlsx',
 'data/Current Year Monthly and Annual Summary Report (1979 - Present) (Nov).xlsx',
 'data/Current Year Monthly and Annual Summary Report (1979 - Present) (Oct).xlsx',
 'data/Current Year Monthly and Annual Summary Report (1979 - Present) (Sep)

In [3]:
nps_df = pd.DataFrame()
nps_df = nps_df.iloc[0:0]

In [4]:
for f in filepath_xlsx:
    nps_data = pd.read_excel(f, 'Current Year Monthly and Annual', header=11, skipfooter=6)
    if len(nps_df) == 0:
        nps_df = nps_df.append(nps_data)
    else:
        nps_df = nps_df.merge(nps_data, how="inner", on="Park")

In [5]:
clean_nps_df = nps_df[['Park'
                       ,'MAR 2021', 'MAR 2020'
                       ,'FEB 2021', 'FEB 2020'
                       ,'JAN 2021', 'JAN 2020'
                       ,'DEC 2020', 'DEC 2019'
                       ,'NOV 2020', 'NOV 2019'
                       ,'OCT 2020', 'OCT 2019'
                       ,'SEP 2020', 'SEP 2019'
                       ,'AUG 2020', 'AUG 2019'
                       ,'JUL 2020', 'JUL 2019'
                       ,'JUN 2020', 'JUN 2019'
                       ,'MAY 2020', 'MAY 2019'
                       ,'APR 2020', 'APR 2019']]

np_designations = clean_nps_df['Park'].str[-2:]
    
clean_nps_df.insert(1, "Designation", np_designations, True)

clean_nps_df.head()

,Park,Designation,MAR 2021,MAR 2020,FEB 2021,FEB 2020,JAN 2021,JAN 2020,DEC 2020,DEC 2019,...,AUG 2020,AUG 2019,JUL 2020,JUL 2019,JUN 2020,JUN 2019,MAY 2020,MAY 2019,APR 2020,APR 2019
0,Abraham Lincoln Birthplace NHP,HP,15362.0,12612.0,5192.0,7406.0,6320.0,6911.0,7472.0,6764.0,...,31290.0,34816.0,37249.0,37334.0,29111.0,26854.0,21102.0,24646.0,15092.0,16496.0
1,Acadia NP,NP,38060.0,28539.0,16123.0,13459.0,20268.0,12640.0,21260.0,12865.0,...,681746.0,759735.0,493971.0,756439.0,201156.0,490720.0,103120.0,226156.0,55713.0,68046.0
2,Adams NHP,HP,521.0,598.0,425.0,320.0,483.0,300.0,544.0,256.0,...,520.0,26011.0,460.0,75333.0,759.0,23963.0,743.0,13187.0,1127.0,537.0
3,African Burial Ground NM,NM,331.0,627.0,35.0,1994.0,198.0,4279.0,116.0,4279.0,...,0.0,4332.0,0.0,5520.0,0.0,4621.0,0.0,5482.0,0.0,5022.0
4,Agate Fossil Beds NM,NM,279.0,176.0,65.0,194.0,94.0,107.0,121.0,50.0,...,1686.0,2775.0,2377.0,4222.0,834.0,3276.0,310.0,2188.0,129.0,391.0


In [6]:
index_to_drop = clean_nps_df[ clean_nps_df["Designation"] != "NP"].index

np_df = clean_nps_df.drop(index_to_drop)

sorted_np_df = np_df.sort_values(["MAR 2021"], ascending=False)

sorted_np_df.reset_index(inplace=True)

sorted_np_df["visit_rank"] = np.arange(len(sorted_np_df))+1

clean_np_df = sorted_np_df.drop(["index", "Designation"], axis=1)

clean_np_df.head()

,Park,MAR 2021,MAR 2020,FEB 2021,FEB 2020,JAN 2021,JAN 2020,DEC 2020,DEC 2019,NOV 2020,...,AUG 2019,JUL 2020,JUL 2019,JUN 2020,JUN 2019,MAY 2020,MAY 2019,APR 2020,APR 2019,visit_rank
0,Great Smoky Mountains NP,931060.0,487535.0,502459.0,426839.0,525801.0,438509.0,777584.0,720631.0,1091098.0,...,1411231.0,1726187.0,1613133.0,1535660.0,1469857.0,828792.0,1096056.0,99371.0,999259.0,1
1,Zion NP,429455.0,207400.0,160232.0,133718.0,172747.0,126994.0,227244.0,171706.0,319853.0,...,535322.0,449518.0,629802.0,377425.0,594896.0,178194.0,529553.0,40396.0,372349.0,2
2,Joshua Tree NP,375049.0,226600.0,285687.0,298294.0,229995.0,268187.0,268640.0,303476.0,360238.0,...,120911.0,147037.0,138967.0,184139.0,187551.0,89831.0,250928.0,0.0,374789.0,3
3,Grand Canyon NP,320277.0,276002.0,163115.0,252897.0,166122.0,246460.0,245830.0,324380.0,275764.0,...,700896.0,350003.0,756860.0,232267.0,643821.0,43397.0,618598.0,0.0,562265.0,4
4,Saguaro NP,200968.0,98901.0,177726.0,146224.0,186428.0,124069.0,163339.0,96235.0,52377.0,...,51309.0,25649.0,48553.0,20362.0,49481.0,22300.0,75838.0,22820.0,117802.0,5


In [7]:
#Rename columns
clean_np_df.rename(columns={'Park': 'name'
                            ,'MAR 2021' : 'mar2021'
                            , 'MAR 2020': 'mar2020'
                            ,'FEB 2021': 'feb2021'
                            , 'FEB 2020': 'feb2020'
                            ,'JAN 2021': 'jan2021'
                            , 'JAN 2020': 'jan2020'
                            ,'DEC 2020': 'dec2020'
                            , 'DEC 2019': 'dec2019'
                            ,'NOV 2020': 'nov2020'
                            , 'NOV 2019': 'nov2019'
                            ,'OCT 2020': 'oct2020'
                            , 'OCT 2019': 'oct2019'
                            ,'SEP 2020': 'sep2020'
                            , 'SEP 2019': 'sep2019'
                            ,'AUG 2020': 'aug2020'
                            , 'AUG 2019': 'aug2019'
                            ,'JUL 2020': 'jul2020'
                            , 'JUL 2019': 'jul2019'
                            ,'JUN 2020': 'jun2020'
                            , 'JUN 2019': 'jun2019'
                            ,'MAY 2020': 'may2020'
                            , 'MAY 2019': 'may2019'
                            ,'APR 2020': 'apr2020'
                            , 'APR 2019': 'apr2019'                           
                           }, inplace=True)

clean_np_df.head()

,name,mar2021,mar2020,feb2021,feb2020,jan2021,jan2020,dec2020,dec2019,nov2020,...,aug2019,jul2020,jul2019,jun2020,jun2019,may2020,may2019,apr2020,apr2019,visit_rank
0,Great Smoky Mountains NP,931060.0,487535.0,502459.0,426839.0,525801.0,438509.0,777584.0,720631.0,1091098.0,...,1411231.0,1726187.0,1613133.0,1535660.0,1469857.0,828792.0,1096056.0,99371.0,999259.0,1
1,Zion NP,429455.0,207400.0,160232.0,133718.0,172747.0,126994.0,227244.0,171706.0,319853.0,...,535322.0,449518.0,629802.0,377425.0,594896.0,178194.0,529553.0,40396.0,372349.0,2
2,Joshua Tree NP,375049.0,226600.0,285687.0,298294.0,229995.0,268187.0,268640.0,303476.0,360238.0,...,120911.0,147037.0,138967.0,184139.0,187551.0,89831.0,250928.0,0.0,374789.0,3
3,Grand Canyon NP,320277.0,276002.0,163115.0,252897.0,166122.0,246460.0,245830.0,324380.0,275764.0,...,700896.0,350003.0,756860.0,232267.0,643821.0,43397.0,618598.0,0.0,562265.0,4
4,Saguaro NP,200968.0,98901.0,177726.0,146224.0,186428.0,124069.0,163339.0,96235.0,52377.0,...,51309.0,25649.0,48553.0,20362.0,49481.0,22300.0,75838.0,22820.0,117802.0,5


In [8]:
nps_url = f"https://developer.nps.gov/api/v1/parks?limit=500&api_key={nps_api_key}"

nps_url

'https://developer.nps.gov/api/v1/parks?limit=500&api_key=yY2lhkKaW1Sq0mBMXP34xubhI9gndEP31UqIPyEf'

In [9]:
response = requests.get(nps_url)
json_response = response.json()

parks = json_response["data"]

In [10]:
names = []
lats = []
longs = []
urls = []
descriptions = []


for park in parks:
    if (park["designation"] == "National Park" or park["designation"] == "National Parks" or park["designation"] == "National and State Parks"):
        park_name = park["name"]
        names.append(f'{park_name} NP')
        lats.append(park["latitude"])
        longs.append(park["longitude"])
        urls.append(park["url"])
        descriptions.append(park["description"])
    
    

In [11]:
# create dataframe
np_details_df = pd.DataFrame({
    "name": names,
    "description": descriptions,
    "latitude": lats,
    "longitude": longs,
    "url": urls
})

np_details_df.head()

,name,description,latitude,longitude,url
0,Acadia NP,Acadia National Park protects the natural beau...,44.409286,-68.247501,https://www.nps.gov/acad/index.htm
1,Arches NP,Visit Arches to discover a landscape of contra...,38.72261844,-109.5863666,https://www.nps.gov/arch/index.htm
2,Badlands NP,The rugged beauty of the Badlands draws visito...,43.68584846,-102.482942,https://www.nps.gov/badl/index.htm
3,Big Bend NP,There is a place in Far West Texas where night...,29.29817767,-103.2297897,https://www.nps.gov/bibe/index.htm
4,Biscayne NP,"Within sight of downtown Miami, yet worlds awa...",25.490587,-80.21023851,https://www.nps.gov/bisc/index.htm


In [12]:
final_nps_df = np_details_df.merge(clean_np_df, how="inner", on="name")

final_nps_df.columns

Index(['name', 'description', 'latitude', 'longitude', 'url', 'mar2021',
       'mar2020', 'feb2021', 'feb2020', 'jan2021', 'jan2020', 'dec2020',
       'dec2019', 'nov2020', 'nov2019', 'oct2020', 'oct2019', 'sep2020',
       'sep2019', 'aug2020', 'aug2019', 'jul2020', 'jul2019', 'jun2020',
       'jun2019', 'may2020', 'may2019', 'apr2020', 'apr2019', 'visit_rank'],
      dtype='object')

In [13]:
#Load Database
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
conn = engine.connect()

In [14]:
engine.table_names()

['park_detail', 'park_visits', 'parks']

In [15]:
engine.execute("delete from parks")

In [16]:
#Load parks table
final_nps_df.to_sql(name="parks", con=engine, if_exists='append', index=False)

In [17]:
#Confirm data was loaded into parks table
pd.read_sql_query('select * from parks', con=engine)

,parks_id,name,description,latitude,longitude,url,mar2021,mar2020,feb2021,feb2020,...,aug2019,jul2020,jul2019,jun2020,jun2019,may2020,may2019,apr2020,apr2019,visit_rank
0,50,Acadia NP,Acadia National Park protects the natural beau...,44.409286,-68.247501,https://www.nps.gov/acad/index.htm,38060,28539,16123,13459,...,759735,493971,756439,201156,490720,103120,226156,55713,68046,27
1,51,Arches NP,Visit Arches to discover a landscape of contra...,38.722618,-109.586367,https://www.nps.gov/arch/index.htm,81623,81623,53986,42416,...,184890,193824,208993,163699,225206,13832,217474,0,168788,19
2,52,Badlands NP,The rugged beauty of the Badlands draws visito...,43.685848,-102.482942,https://www.nps.gov/badl/index.htm,28182,17183,11350,9979,...,207743,238875,250898,167784,201516,55637,71885,17290,25083,34
3,53,Big Bend NP,There is a place in Far West Texas where night...,29.298178,-103.229790,https://www.nps.gov/bibe/index.htm,89915,61413,43889,50584,...,17238,0,21747,26735,21228,0,37411,0,49312,17
4,54,Biscayne NP,"Within sight of downtown Miami, yet worlds awa...",25.490587,-80.210239,https://www.nps.gov/bisc/index.htm,22662,11052,19596,91655,...,46191,28770,60239,32646,65173,43692,79779,2358,64172,36
5,55,Bryce Canyon NP,Hoodoos (irregular columns of rock) exist on e...,37.583991,-112.182669,https://www.nps.gov/brca/index.htm,96800,45132,35653,38890,...,383458,227563,401970,195463,398290,69579,327975,3223,207061,15
6,56,Canyonlands NP,Canyonlands invites you to explore a wildernes...,38.245558,-109.880162,https://www.nps.gov/cany/index.htm,87001,14490,20164,12120,...,69297,70480,64672,63685,92295,6304,114464,0,89861,18
7,57,Capitol Reef NP,Located in south-central Utah in the heart of ...,38.282165,-111.247048,https://www.nps.gov/care/index.htm,115728,45804,25718,20579,...,127341,123274,129372,158421,151410,90903,196840,13014,147156,13
8,58,Carlsbad Caverns NP,"High ancient sea ledges, deep rocky canyons, f...",32.140895,-104.552969,https://www.nps.gov/cave/index.htm,30749,28925,13270,20860,...,36354,18855,69444,10124,57111,1338,41516,1138,37543,32
9,59,Channel Islands NP,Channel Islands National Park encompasses five...,33.986801,-119.911273,https://www.nps.gov/chis/index.htm,5611,13408,5815,20318,...,49793,15323,47581,13740,44915,2514,39070,365,39218,47
